# Import Useful Modules 

In [1]:
import tensorflow
config = tensorflow.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 #(misalnya kalo pengen 0.4 dari GPU memory)
session = tensorflow.Session(config=config)

D:\Program\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import gmtime, strftime
import time
import datetime
from collections import Counter
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# # Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Embedder
from gensim.models import FastText

# Classifier
# from sklearn.svm import SVC
# from sklearn.ensemble import GradientBoostingClassifier as GBC
# from sklearn.linear_model import LogisticRegression as LR
# from sklearn.ensemble import RandomForestClassifier as RFC
# from sklearn.grid_search import GridSearchCV as GS
from xgboost import XGBClassifier as XGB
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.preprocessing import OneHotEncoder as OHE

# import keras
# import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical
from keras import regularizers

from preprocessing_pipeline import preprocessing

D:\Program\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [3]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
import tqdm

In [5]:
#Determine Model's Save Location

#version=
version="version_x"

In [6]:
if(not(os.path.exists("model/{}".format(version)))):
    os.makedirs("model/{}".format(version))

In [7]:
from preprocessing_pipeline import preprocessing

with open("model/{}/word_embedder.pickle".format(version), "rb") as file:
    word_embedder = pickle.load(file)

In [8]:
preprocessor=preprocessing(word_embedder.vector_size,word_embedder)
preprocessor

# ---------------------------------------------------------------------------------------------------------------

# Read Data To Pandas Dataframe

In [9]:
def read_product_title_data():
    product_title_only=pd.read_csv('data/query.csv',header=None)
    product_title_only.dropna(inplace=True,axis=0)
    features=product_title_only[1]
    labels=product_title_only[0]
    
    
#     return (product_title_only[:300000],
#             product_title_only[300000:600000],
#             product_title_only[600000:900000],
#             product_title_only[900000:1200000],
#             product_title_only[1200000:1500000],
#             product_title_only[1500000:1800000],
#             product_title_only[1800000:])
    
    return features,labels

In [10]:
def read_product_title_data_small():
    product_title_only=pd.read_csv('data/big.csv',header=None)
    product_title_only.dropna(inplace=True,axis=0)
    features=product_title_only[1]
    labels=product_title_only[0]
    
#     return (product_title_only[:300000],
#             product_title_only[300000:600000],
#             product_title_only[600000:900000],
#             product_title_only[900000:1200000],
#             product_title_only[1200000:1500000],
#             product_title_only[1500000:1800000],
#             product_title_only[1800000:])
    
    return features,labels

In [11]:
def word_count(sentences):
    counts = dict()
    print("1/1")
    for sentence in sentences:
        for word in sentence:
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
    return counts

def class_count(words):
    counts = dict()
    print("1/1")
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

def getFilteredData(product_title,labels,frequency, N_words, word_length):
    print("1/3")
    result=word_count(product_title)
    print("2/3")
    new_product_title=[]
    for sentence in tqdm.tqdm(product_title):
        new_product_title.append([word for word in sentence if result[word]>=frequency and len(word)>=word_length])
    
    print("3/3")
    new_features=[]
    new_labels=[]
    for index,title in tqdm.tqdm(enumerate(new_product_title)):
        if(len(title)>=N_words):
            new_features.append(title)
            new_labels.append(labels[index])
    
    return new_features,new_labels

def getTfIdf(new_product_title):
    print("1/3")
    concatenated_product_title=[]
    for sentence in tqdm.tqdm(new_product_title):
        concatenated_product_title.append(" ".join(sentence))
    print("2/3")
    cv=CountVectorizer()
    result=cv.fit_transform(concatenated_product_title)
    print("3/3")
    tftransformer = TfidfTransformer(smooth_idf=False)
    final_result=tftransformer.fit_transform(result)
    
    return final_result,cv,tftransformer

def getTfIdfCustom(new_product_title,vocab):
    print("1/3")
    concatenated_product_title=[]
    for sentence in tqdm.tqdm(new_product_title):
        concatenated_product_title.append(" ".join(sentence))
    print("2/3")
    cv=CountVectorizer(vocabulary=vocab)
    result=cv.fit_transform(concatenated_product_title)
    print("3/3")
    tftransformer = TfidfTransformer(smooth_idf=False)
    final_result=tftransformer.fit_transform(result)
    
    return final_result,cv,tftransformer

def getFilteredClasses(product_title,labels,top_N):
    print("1/3")
    sorted_by_value = sorted(class_count(labels).items(), key=lambda kv: kv[1])
    valid_class=[value[0] for value in sorted_by_value[-top_N:]]
    print("2/3")
    product_title=list(product_title)
    new_features=[]
    new_labels=[]
    for index,label in tqdm.tqdm(enumerate(labels)):
        if(label in valid_class):
            new_labels.append(label)
            new_features.append(product_title[index])
    
    return new_features,new_labels

In [12]:
def word_count_label(sentences,labels,target):
    counts = dict()
    print("1/1")
    for index,sentence in enumerate(sentences):
        if(labels[index]==target):
            for word in sentence:
                if word in counts:
                    counts[word] += 1
                else:
                    counts[word] = 1
    return counts

In [13]:
data_for_embedding=read_product_title_data_small()

In [14]:
sorted_by_value = sorted(class_count(data_for_embedding[1]).items(), key=lambda kv: kv[1])
valid_class=np.sum([value[1] for value in sorted_by_value[-30:]])
print(valid_class/len(data_for_embedding[0]))

1/1
0.6002772995416855


In [16]:
nfnf,nlnl=getFilteredClasses(data_for_embedding[0],data_for_embedding[1],300)

1/3
1/1


0it [00:00, ?it/s]

2/3


2440682it [00:15, 157900.71it/s]


#  ---------------------------------------------------------------------------------------------------

In [34]:
product_title=[preprocessor.remove_parentheses(value) for value in tqdm.tqdm(nfnf)]

100%|██████████| 1465086/1465086 [00:17<00:00, 84487.07it/s]


In [35]:
new_data=getFilteredData(product_title,list(nlnl),50,2,3)

1/3
1/1


  1%|          | 15967/1465086 [00:00<00:41, 34596.93it/s]

2/3


100%|██████████| 1465086/1465086 [00:04<00:00, 328552.05it/s]
203679it [00:00, 1018160.51it/s]

3/3


1465086it [00:00, 1620229.29it/s]


In [36]:
fff=new_data[0]
lll=new_data[1]

top_3={}
for label in tqdm.tqdm(set(lll)):
    top_3[label]=[value[0] for value in sorted(word_count_label(fff,lll,label).items(), key=lambda x: x[1])[-30:]]
    
allw=[]
for label in list(top_3):
    for i in top_3[label]:
        allw.append(i)
        

  3%|▎         | 1/30 [00:00<00:05,  5.19it/s]

1/1
1/1


 10%|█         | 3/30 [00:00<00:04,  6.15it/s]

1/1
1/1


 17%|█▋        | 5/30 [00:00<00:04,  5.96it/s]

...

 87%|████████▋ | 26/30 [00:03<00:00,  7.00it/s]

1/1
1/1
1/1


 97%|█████████▋| 29/30 [00:04<00:00,  7.08it/s]

1/1
1/1


100%|██████████| 30/30 [00:04<00:00,  7.03it/s]


In [37]:
aaaaa=getTfIdfCustom(new_data[0],set(allw))

 33%|███▎      | 343091/1030967 [00:00<00:00, 1715104.97it/s]

1/3


100%|██████████| 1030967/1030967 [00:00<00:00, 1515835.48it/s]


2/3
3/3


In [38]:
newnew=[]
for index,value in tqdm.tqdm(enumerate(aaaaa[0].toarray())):
    temp=list(value)+list(preprocessor.vectorize_sentence(new_data[0][index]))
    newnew.append(temp) 

1030967it [03:23, 5074.99it/s]


In [39]:
llee=LabelEncoder()
wenwen=llee.fit_transform(lll)

# ---------------------------------------------------------------------------------------------------------------

In [40]:
nn_X_train,nn_X_test,nn_y_train,nn_y_test=train_test_split(newnew,to_categorical(wenwen),test_size=0.1)

In [43]:
model = Sequential()
model.add(Dense(1000, input_shape=(len(nn_X_train[0]),), activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(len(nn_y_train[0]), activation='softmax'))


model.compile(optimizer="Adagrad", loss='categorical_crossentropy',metrics=['accuracy'])

In [44]:
history = model.fit([nn_X_train], nn_y_train, initial_epoch=0, epochs=25, batch_size=100, validation_data=([nn_X_test],nn_y_test), shuffle=True)

Train on 927870 samples, validate on 103097 samples
Epoch 1/25
927870/927870 [==============================] - 34s 37us/step - loss: 0.5808 - acc: 0.8321 - val_loss: 0.5025 - val_acc: 0.8504
Epoch 2/25
927870/927870 [==============================] - 33s 36us/step - loss: 0.4546 - acc: 0.8620 - val_loss: 0.4589 - val_acc: 0.8611
Epoch 3/25
927870/927870 [==============================] - 34s 37us/step - loss: 0.4090 - acc: 0.8725 - val_loss: 0.4363 - val_acc: 0.8668
Epoch 4/25
927870/927870 [==============================] - 34s 37us/step - loss: 0.3795 - acc: 0.8789 - val_loss: 0.4222 - val_acc: 0.8706
Epoch 5/25
927870/927870 [==============================] - 40s 43us/step - loss: 0.3588 - acc: 0.8835 - val_loss: 0.4101 - val_acc: 0.8724
Epoch 6/25
927870/927870 [==============================] - 43s 47us/step - loss: 0.3426 - acc: 0.8871 - val_loss: 0.4022 - val_acc: 0.8740
Epoch 7/25
927870/927870 [==============================] - 43s 46us/step - loss: 0.3299 - acc: 0.8901 - val

KeyboardInterrupt: 

In [39]:
pred=model.predict(nn_X_test)

KeyboardInterrupt: 

In [55]:
print(classification_report(model.predict_classes([nn_X_test]),[np.argmax(value) for value in nn_y_test]))

             precision    recall  f1-score   support

          0       0.52      0.65      0.58       302
          1       0.04      0.83      0.08         6
          2       0.73      0.54      0.62      7291
          3       0.78      0.68      0.72      8984
          4       0.34      0.35      0.35       420
          5       0.72      0.57      0.63      7886
          6       0.67      0.61      0.64      8456
          7       0.45      0.47      0.46       535
          8       0.21      0.50      0.30       383
          9       0.46      0.54      0.50       397
         10       0.51      0.55      0.53       800
         11       0.43      0.48      0.45       658
         12       0.58      0.60      0.59       742
         13       0.35      0.45      0.40       444
         14       0.70      0.65      0.68      1182
         15       0.08      1.00      0.15         4
         16       0.81      0.86      0.83       527
         17       0.53      0.47      0.50   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# ---------------------------------------------------------------------------------------------------------------

In [17]:
#preprocess product title to 100-dimensional vector
#and preprocess category name to integer label
large_embedded_data, large_label_encoder = preprocessor.preprocess_data(
    nfnf,
    nlnl,
)

  0%|          | 147/2440682 [00:00<28:29, 1427.43it/s]

REMOVING UNIMPORTANT CHARACTERS


100%|██████████| 2440682/2440682 [00:34<00:00, 71163.47it/s]


APPLYING FILTER


100%|██████████| 2440682/2440682 [00:07<00:00, 310338.14it/s]
2440682it [00:01, 1439238.89it/s]


...

  0%|          | 490/1757951 [00:00<05:59, 4894.85it/s]

CONVERTING SENTENCE TO VECTOR


  0%|          | 0/300 [00:00<?, ?it/s]

SAVE VECTOR TO PANDAS DATAFRAME


100%|██████████| 300/300 [05:02<00:00,  1.01s/it]


In [18]:
large_embedded_data.head()
large_embedded_data["sum"]=large_embedded_data.drop(["Labels"],axis=1).sum(axis=1)
large_embedded_data=large_embedded_data.loc[large_embedded_data["sum"]!=0].drop("sum",axis=1)

In [19]:
nn_X_train,nn_X_test,nn_y_train,nn_y_test=train_test_split(large_embedded_data.drop("Labels",axis=1),to_categorical(large_embedded_data["Labels"]),test_size=0.1)

In [35]:
model = Sequential()
model.add(Dense(1000, input_shape=(300,), activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(232, activation='softmax'))


model.compile(optimizer="Adagrad", loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(nn_X_train, nn_y_train, initial_epoch=0, epochs=25, batch_size=100, validation_data=(nn_X_test,nn_y_test), shuffle=True)

Train on 1582081 samples, validate on 175787 samples
Epoch 1/25
